<a href="https://colab.research.google.com/github/snikhil17/NLP_course_Simplilearn/blob/main/Class_Codes_and_Assignments/Notebooks/5.%20Nord_NIkhil_Shrestha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textblob

In [ ]:
!pip install langdetect

In [ ]:
!pip install pyLDAvis

In [ ]:
import pandas as pd
import numpy as np

#Visualizing the LDA model
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packa

In [155]:
df = pd.read_csv('nord.csv')
df.head()

,title,content,date,verified,author,rating
0,Finally Settled for Average Performance and Ba...,"For 25K , you will get - Average performance. ...",30-Sep-20,Yes,arv,2
1,"6gb+64gb not worth buying, storage issues are ...","The phone is really great performance wise, th...",26-Sep-20,Yes,Shivani,2
2,Perfect phone for light to moderate usage,I have been using it since 14-15 days. So here...,27-Sep-20,Yes,Simranjit Singh,5
3,Love this phone from one plus,Such a nice phone from one plus.over all it's ...,26-Sep-20,Yes,Rohit Sahu,4
4,"Never a oneplus standard in terms of sale, and...",Regarding the one plus feeling. It's gone.. Se...,30-Sep-20,Yes,Amazon Customer,2


In [156]:
df.shape

(707, 6)

In [157]:
# Dropping NA
df.dropna(inplace = True)

## **Detecting English Languages and keeping only those rows**

In [158]:
from langdetect import detect

df['content'] = df['content'].apply(lambda x: x if (detect(x) == 'en') else np.nan)

df.dropna(inplace = True)

df.content

0      For 25K , you will get - Average performance. ...
1      The phone is really great performance wise, th...
2      I have been using it since 14-15 days. So here...
3      Such a nice phone from one plus.over all it's ...
4      Regarding the one plus feeling. It's gone.. Se...
                             ...                        
676                                             Used ine
678                         Battery not working properly
703                                             Very bad
705         Worst product. Nevar purchase renewd product
706    I have purchased 10 feb 2021 and insert sim to...
Name: content, Length: 531, dtype: object

## Decontract words:
- Don't -> Do not

In [159]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

df['content'] = df['content'].apply(lambda x: decontracted(x))

## **Work with ``Content`` variable**

In [160]:
# Remove words like 'nor', 'not', 'no' from stop_words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')

for i in ['nor', 'not', 'no']:
  stop_words.remove(i)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [161]:
#Create Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(stop_words=stop_words, #Stopwords
                              lowercase=True, # Normalization
                              token_pattern= r'\b[a-zA-Z]{3,}', #Tokenization
                                strip_accents='unicode',  #Trims all unicode chars which is not a part of eng language
                              )

tfObject = tf_vectorizer.fit_transform(df['content'])
tfObject.shape


(531, 1586)

In [162]:
#Create TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
tfidfObject = tfidfVectorizer.fit_transform(df['content'])
tfidfObject.shape

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:2032: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning,


(531, 1586)

## **Aprroach:**
- Using 4 components for BOW and 3 components for TF-IDF
  - Positive, 
  - Negative, 
  - Battery and display, 
  - Money Internet and Support etc.
- 3: Good Bad and Average

In [163]:
#Apply LDA to create topic clusters for Bag of Words

from sklearn.decomposition import LatentDirichletAllocation
lda_tf = LatentDirichletAllocation(n_components=4 , random_state=0)
lda_tf.fit(tfObject) # as per books

LatentDirichletAllocation(n_components=4, random_state=0)

In [164]:
from sklearn.decomposition import LatentDirichletAllocation
lda_tfIDF = LatentDirichletAllocation(n_components=3 , random_state=0)
lda_tfIDF.fit(tfidfObject) # as per seasoned DS considering memory optimizn reqd for handling Bigdata

LatentDirichletAllocation(n_components=3, random_state=0)

In [165]:
#---------------------LDAModel,BOW/TFIDF Object,CountVectorizerObject ----------

pyLDAvis.sklearn.prepare(lda_tf,tfObject,tf_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.042811  0.042095       1        1  39.019460
2      0.079423  0.035474       2        1  36.830004
0      0.004644 -0.109954       3        1  18.285097
1     -0.126877  0.032385       4        1   5.865439, topic_info=           Term        Freq       Total Category  logprob  loglift
1071    product   67.000000   67.000000  Default  30.0000  30.0000
896        nice   31.000000   31.000000  Default  29.0000  29.0000
1001      phone  219.000000  219.000000  Default  28.0000  28.0000
141        best   47.000000   47.000000  Default  27.0000  27.0000
600        good  182.000000  182.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
400     display    1.412739   48.036332   Topic4  -5.6621  -0.6903
368   defective    1.267813    7.820193   Topic4  -5.7704   1.0167
1037       poor    1.259426   11.254743   Topic4  -5.7770   0.6460
385      device    1.256975   18.919701   Topic4  -5.7790   0.1246
1224     screen    1.256603   29.806951   Topic4  -5.7793  -0.3302

[295 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3         2  0.913975  absolutely
36        1  0.674077        also
36        2  0.234461        also
36        3  0.058615        also
36        4  0.058615        also
...     ...       ...         ...
1565      2  0.627444       worth
1565      3  0.048265       worth
1565      4  0.048265       worth
1575      3  0.865715       wrong
1580      1  0.891605         yes

[425 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2])

## Observation:
- Above Vizz is from BOW corpus
- 1st Topic Overall sentiments (anticipating Negative)
- 2nd Topic is somewhat good reviews about battery and display.
- 3rd Topic is for Positive reviews about Value for money, Internet, support.
- 4th Topic is OVerall positive reviews.

In [166]:
pyLDAvis.sklearn.prepare(lda_tfIDF,tfidfObject,tf_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.070960  0.009556       1        1  51.009031
0      0.022202 -0.043078       2        1  35.882615
1      0.048758  0.033522       3        1  13.108354, topic_info=           Term       Freq      Total Category  logprob  loglift
600        good  23.000000  23.000000  Default  30.0000  30.0000
896        nice   6.000000   6.000000  Default  29.0000  29.0000
1099    quality  10.000000  10.000000  Default  28.0000  28.0000
221      camera  13.000000  13.000000  Default  27.0000  27.0000
1071    product  12.000000  12.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
1483       user   0.453032   2.216830   Topic3  -6.0657   0.4441
1477       upto   0.325094   0.966043   Topic3  -6.3976   0.9428
1161    renewed   0.415632   2.388078   Topic3  -6.1519   0.2835
368   defective   0.349684   2.109413   Topic3  -6.3247   0.2348
818        mark   0.313418   1.598957   Topic3  -6.4341   0.4024

[176 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
40        1  0.872555  amazing
40        2  0.218139  amazing
41        1  0.916869   amazon
101       1  0.745185  average
104       1  0.944076  awesome
...     ...       ...      ...
1541      1  1.117379     west
1560      1  0.939111  working
1564      1  0.961023    worst
1565      1  0.403613    worth
1565      2  0.605420    worth

[147 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

## Observation:
- Above Vizz is from BOW corpus
- 1st Topic clearly shows that reviews were bad. 
- 2nd Topic is somewhat good reviews.
- 3rd Topic contains Average Reviews

In [167]:
topic_results = lda_tfIDF.transform(tfObject)

In [168]:
(topic_results.shape)

(531, 3)

In [169]:
topic_results

array([[0.59723849, 0.00222667, 0.40053484],
       [0.10314274, 0.01093839, 0.88591886],
       [0.58401128, 0.00548259, 0.41050613],
       ...,
       [0.1774297 , 0.16699806, 0.65557224],
       [0.04842441, 0.32233058, 0.62924501],
       [0.00855811, 0.00789362, 0.98354827]])

In [170]:
final_topics = []
for i in range(df.shape[0]):
  final_topics.append(topic_results[i].argmax())

In [172]:
df['topics_BOW'] = final_topics
df['topics_BOW'] = df.topics_BOW.replace({0: "Negative", 1: "Positive", 2: "Neutral" })

In [174]:
df['topics_tf_idf'] = final_topics
df['topics_tf_idf'] = df.topics_tf_idf.replace({0: "Negative", 1: "Battery_display", 2: "Money_Internet_Support",3: "Positive" })

In [175]:
df.head()

,title,content,date,verified,author,rating,topics_BOW,topics_tf_idf
0,Finally Settled for Average Performance and Ba...,"For 25K , you will get - Average performance. ...",30-Sep-20,Yes,arv,2,Negative,Negative
1,"6gb+64gb not worth buying, storage issues are ...","The phone is really great performance wise, th...",26-Sep-20,Yes,Shivani,2,Neutral,Money_Internet_Support
2,Perfect phone for light to moderate usage,I have been using it since 14-15 days. So here...,27-Sep-20,Yes,Simranjit Singh,5,Negative,Negative
3,Love this phone from one plus,Such a nice phone from one plus.over all it is...,26-Sep-20,Yes,Rohit Sahu,4,Neutral,Money_Internet_Support
4,"Never a oneplus standard in terms of sale, and...",Regarding the one plus feeling. It is gone.. S...,30-Sep-20,Yes,Amazon Customer,2,Negative,Negative
